In [1]:
import pickle
import requests
import json
import pickle
import time
import random
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def getdemo(url):
    news_id = url.split("/")[4]
    article_id = url.split('/')[5].split('?')[0]
    
    api_url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_cv=20231024153951&_callback=jQuery331007496211539142106_1698626557419&lang=ko&country=KR&objectId=news{news_id}%2C{article_id}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=favorite&includeAllStatus=true&_=1698626557421"
    
    demoinfo = {}
    
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.39.132 Safari/53.36',
           'Referer':  api_url}

    r = requests.get(api_url, headers=headers)

    raw_data = r.text
    start_index = raw_data.find('(') + 1
    main_data = raw_data[start_index:-2]
    main_data_dict = json.loads(main_data)

    try:
        # 성별 정보 저장
        demoinfo['male'] = main_data_dict['result']['graph']['gender']['male']
        demoinfo['female'] = main_data_dict['result']['graph']['gender']['female']

        # 연령대별 정보 저장
        ages = ["10", "20", "30", "40", "50", "60"]
        for i, age in enumerate(ages):
            demoinfo[f'age_{age}'] = main_data_dict['result']['graph']['old'][i]['value']
    except:
        return {}
    
    return demoinfo

In [3]:
data = pd.read_csv('../data_crawling/news_final/mbc_final.csv', index_col=0)

# 3월 데이터만 가져오기
mbc3 = data[data['time'].str.startswith(('2023-03'))]
# 중복기사 삭제
mbc3 = mbc3.drop_duplicates(subset='title', keep='first')

# index 만들기
mbc3['index'] = range(len(mbc3))

# 'index' 칸을 DataFrame의 인덱스로 지정
mbc3 = mbc3.set_index('index')
mbc3.index = mbc3.index+1
mbc3

,title,link,time,contents,category
index,,,,,
1,"윤 대통령 ""일본, 안보·경제 파트너""",https://n.news.naver.com/article/214/000125695...,2023-03-01 12:37:45,"[정오뉴스]◀ 앵커 ▶104주년 3.1절 기념식에 참석한 윤석열 대통령은 ""일본은 ...",정치
2,"윤 대통령, 첫 3.1절 기념사서 ""일본은 협력 파트너""‥양국 현안은 언급 안해",https://n.news.naver.com/article/214/000125700...,2023-03-01 19:45:10,"[뉴스데스크]◀ 앵커 ▶윤석열 대통령은 3.1절 기념식에서 ""일본은 과거 군국주의 ...",정치
3,"민주 ""윤 대통령의 3·1절 기념사‥일본 식민지배 정당화 논리""",https://n.news.naver.com/article/214/000125697...,2023-03-01 14:40:37,자료사진 더불어민주당이 일본과의 가치 공유와 협력을 강조한 윤석열 대통령의 3·1절...,정치
4,'배신자 색출' 주장까지‥내홍 휩싸인 민주당,https://n.news.naver.com/article/214/000125687...,2023-03-01 06:03:58,[뉴스투데이]◀ 앵커 ▶이재명 대표 체포동의안 표결의 후폭풍이 민주당을 덮쳤습니다....,정치
5,"윤 대통령, 3·1절 기념사에서 ""자유, 번영"" 강조",https://n.news.naver.com/article/214/000125692...,2023-03-01 08:45:55,"윤석열 대통령은 오늘 취임 뒤 첫 3·1절을 맞아 ""자유, 헌신, 기억, 미래, 번...",정치
...,...,...,...,...,...
231,윤석열 대통령 지지율 30%‥작년 11월 이후 최저치 [한국갤럽],https://n.news.naver.com/article/214/000126333...,2023-03-31 14:56:31,윤석열 대통령의 국정 지지율이 30%라는 여론조사 결과가 나왔습니다.한국갤럽이 지난...,정치
232,"대통령실, 이틀째 ""후쿠시마 수산물, 국내 들어올 일 없다""",https://n.news.naver.com/article/214/000126334...,2023-03-31 15:45:57,"자료사진대통령실은 이틀째 ""후쿠시마산 수산물이 국내에 들어올 일은 없다""고 밝혔습니...",정치
233,"블랙핑크 미국 국빈만찬서 공연?‥대통령실 ""방미 일정에 없다""",https://n.news.naver.com/article/214/000126332...,2023-03-31 14:44:45,대통령실 [자료사진]대통령실은 4월말 미국을 국빈 방문하는 윤석열 대통령 공식 만찬...,정치


In [4]:
for i, link in tqdm(enumerate(mbc3['link'], 1), total=len(mbc3['link'])):
    try:
        demoinfo = getdemo(link)

        # demoinfo를 3월 기사에 추가
        for key, value in demoinfo.items():
            mbc3.loc[i, key] = value  # 해당 인덱스의 새로운 컬럼에 값을 할당
        
        mbc3.to_csv('../data_crawling/news_final/mbc3_add_demo.csv', index=False)
        
        print(f"Saved demography for link {i}")
        
        time.sleep(random.random()*1.5)
    
    except Exception as e:
        print(f"Failed to save demography for link {i} due to {e}")

  0%|          | 0/235 [00:00<?, ?it/s]

Saved demography for link 1


  0%|          | 1/235 [00:00<03:18,  1.18it/s]

Saved demography for link 2


  1%|▏         | 3/235 [00:02<02:43,  1.42it/s]

...

 99%|█████████▉| 233/235 [03:57<00:01,  1.22it/s]

Saved demography for link 234


100%|█████████▉| 234/235 [03:58<00:00,  1.11it/s]

Saved demography for link 235


100%|██████████| 235/235 [03:59<00:00,  1.02s/it]


In [5]:
mbc3.isna().sum()

title        0
link         0
time         0
contents     0
category     0
male        10
female      10
age_10      10
age_20      10
age_30      10
age_40      10
age_50      10
age_60      10
dtype: int64

In [6]:
mbc3[mbc3['male'].isna()]

,title,link,time,contents,category,male,female,age_10,age_20,age_30,age_40,age_50,age_60
index,,,,,,,,,,,,,
27,"이재명 ""내부 공격 중단해달라‥상대 진영이 바라는 일""",https://n.news.naver.com/article/214/000125757...,2023-03-04 10:32:35,더불어민주당 이재명 대표가 체포동의안 표결 이후 강성 지지층을 중심으로 '비명계'에...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,국민의힘 새 지도부 투표 첫날 투표율 34.72%‥역대 최고치,https://n.news.naver.com/article/214/000125760...,2023-03-04 18:34:39,자료사진국민의힘 차기 지도부를 뽑는 투표 첫날 투표율이 34.72%로 집계됐습니다....,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,국민의힘 전당대회 모바일 투표율 47.51%‥2021년 투표율 넘어서,https://n.news.naver.com/article/214/000125769...,2023-03-05 17:51:03,TV토론 기념촬영하는 국민의힘 당대표 후보들 [국회사진기자단]국민의힘 차기 지도부를...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,정부가 제시한 피해배상안 배경과 전망은?,https://n.news.naver.com/article/214/000125770...,2023-03-05 20:05:13,[뉴스데스크]◀ 앵커 ▶'일제 강제 동원 피해' 배상 문제를 두고 그동안 우리와 일...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,"북한, 서해 방향으로 단거리 탄도미사일 1발 발사",https://n.news.naver.com/article/214/000125868...,2023-03-09 19:49:09,합동참모본부는 오늘 오후 6시 20분쯤 북한 남포 일대에서 서해 방향으로 발사된 단...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,"""강압수사 때문"" 검찰 책임 주장했지만 내심 곤혹",https://n.news.naver.com/article/214/000125891...,2023-03-10 23:24:43,"[뉴스데스크]◀ 앵커 ▶민주당은, 이재명 대표를 잡으려는 강압적인 수사 때문에 사람...",정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,"당정 ""일제강제징용 배상 해법, 피해자·유가족 소통 지속""",https://n.news.naver.com/article/214/000126062...,2023-03-19 18:38:35,강민국 국민의힘 수석대변인국민의힘과 정부는 일제 강제징용 배상 해법은 피해자와 유족...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,"북한, 탄도미사일 발사‥美 전략폭격기 노렸나",https://n.news.naver.com/article/214/000126063...,2023-03-19 20:05:19,[뉴스데스크]◀ 앵커 ▶북한이 오늘 오전 동해상으로 단거리 탄도미사일 한 발을 발사...,정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,"은밀히 접근해 방사능 해일? 북한, '핵드론' 훈련 공개",https://n.news.naver.com/article/214/000126188...,2023-03-24 19:55:07,"[뉴스데스크]◀ 앵커 ▶북한이 11년 전부터 개발해온 비밀 병기라면서, 핵폭탄을 장...",정치,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
